<a href="https://colab.research.google.com/github/rohailkhan/Data-Science-python/blob/master/2_Project_Multiclass_Classi%0Ccation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# MLP with automatic validation set
from keras.models import Sequential
from keras.optimizers import adam_v2
from keras.layers import Dense,Conv2D, BatchNormalization,MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, Flatten
import numpy as np
import pandas as pd
# fix random seed for reproducibility
seed = 7

np.random.seed(seed)


In [2]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [5]:
path='https://raw.githubusercontent.com/rohailkhan/data/main/iris.csv'
iris=pd.read_csv(path)

In [4]:
iris.head()

,5.1,3.5,1.4,0.2,Iris-setosa
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [6]:
iris=pd.read_csv(path,header=None)
iris.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [11]:
iris.columns

Int64Index([0, 1, 2, 3, 4], dtype='int64')

In [9]:
dataset = iris.values
dataset[0:5]

array([[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.0, 1.4, 0.2, 'Iris-setosa'],
       [4.7, 3.2, 1.3, 0.2, 'Iris-setosa'],
       [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],
       [5.0, 3.6, 1.4, 0.2, 'Iris-setosa']], dtype=object)

In [12]:
X = dataset[:,0:4].astype(float)
Y = dataset[:,4] # last column is the labels

## One hot encoding of the target
This will be done in 2 steps.
1- First the string classes are converted to integers by LabelEncoder

2-Then the converted digits are one-hot encoded using keras .to_categorical

In [21]:
>>> from sklearn import preprocessing
>>> le = preprocessing.LabelEncoder()
>>> le.fit([1, 2, 2, 6])
le.classes_  , le.transform([1, 1, 2, 6])

(array([1, 2, 6]), array([0, 0, 1, 2]))

In [25]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [26]:
from keras.utils.np_utils import to_categorical
one_hot_y=np_utils.to_categorical(encoded_Y)

# Base line Model

In [41]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=4, kernel_initializer='normal', activation='relu'))
    model.add(Dense(3, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Make a wrapper to for Keras Model for use in Scikit-learn

In [42]:
estimator = KerasClassifier(build_fn=baseline_model, nb_epoch=200, batch_size=5, verbose=0)

# Evaluate model using k-fold cross validation

In [43]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

# Results

In [44]:
results = cross_val_score(estimator, X, one_hot_y, cv=kfold)


In [45]:
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 36.00% (11.62%)


# Without kfold

In [54]:
    model = Sequential()
    model.add(Dense(4, input_dim=4, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Flatten())
    model.add(Dense(3,  activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
results=model.fit(X, one_hot_y,epochs=120,shuffle=True)

In [68]:
print("Accuracy:mean= %.2f%% std=%.2f%%" % (np.mean(results.history['accuracy'])*100, np.std(results.history['accuracy'])*100))

Accuracy:mean= 90.78% std=12.68%
